In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers
from os import listdir

2023-08-15 16:53:29.969893: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-15 16:53:32.310468: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-15 16:53:32.311953: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 16:53:43.843491: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data_list_model2c=listdir('train')
data_list_model2c

['COVID', 'bacterial_viral']

In [3]:
DATASET_PATH_model2c='train'
train_dir_model2c='train'
test_dir_model2c='test'


In [4]:
image_size=(150,150)
num_classes=len(data_list_model2c)
batch_size=10
num_epochs=100
learning_rate=0.0001

In [5]:
train_datagen=ImageDataGenerator(
                                rescale=1./255,
                                rotation_range=50,
                                featurewise_center=True,
                                featurewise_std_normalization=True,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.25,
                                zoom_range=0.1,
                                zca_whitening=True,
                                channel_shift_range=20,
                                horizontal_flip=True,
                                vertical_flip=True,
                                fill_mode='constant'
                                )

/home/hpn724/.local/lib/python3.10/site-packages/keras/preprocessing/image.py:1451: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn(


In [6]:
train_batches=train_datagen.flow_from_directory(
                                                DATASET_PATH_model2c,
                                                target_size=image_size,
                                                shuffle=True,
                                                batch_size=batch_size,
                                                subset='training',
                                                seed=42,
                                                class_mode="categorical"
                                                )

Found 175 images belonging to 2 classes.


In [7]:
validate_batches=train_datagen.flow_from_directory(
                                                DATASET_PATH_model2c,
                                                target_size=image_size,
                                                shuffle=True,
                                                batch_size=batch_size,
                                                subset='validation',
                                                seed=42,
                                                class_mode="categorical"
                                                )

Found 0 images belonging to 2 classes.


In [8]:
vgg16_model=tf.keras.applications.vgg16.VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(150,150,3)))

2023-08-15 16:54:00.732311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-15 16:54:00.740428: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [10]:
for layer in vgg16_model.layers[:-4]:
    layer.trainable=False
model2c=tf.keras.Sequential()
model2c.add(vgg16_model)
model2c.add(Flatten())
model2c.add(Dense(64,activation='relu'))
model2c.add(layers.Dense(2,activation='softmax'))

In [11]:
model2c.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),metrics=['accuracy'])

In [12]:
Step_size_train=train_batches.n//train_batches.batch_size
Step_size_valid=validate_batches.n//validate_batches.batch_size

In [13]:
model2c.fit_generator(train_batches,steps_per_epoch=Step_size_train,epochs=5,verbose=True)

/tmp/ipykernel_7701/3817392476.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model2c.fit_generator(train_batches,steps_per_epoch=Step_size_train,epochs=5,verbose=True)
/home/hpn724/.local/lib/python3.10/site-packages/keras/preprocessing/image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/home/hpn724/.local/lib/python3.10/site-packages/keras/preprocessing/image.py:1884: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/5


2023-08-15 16:54:04.164917: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-08-15 16:54:06.234222: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 57600000 exceeds 10% of free system memory.
2023-08-15 16:54:06.572701: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 57600000 exceeds 10% of free system memory.
2023-08-15 16:54:06.703485: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 14400000 exceeds 10% of free system memory.
2023-08-15 16:54:06.715632: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28800000 exceeds 10% of free system memory.
2023-08-15 16:54:06.738308: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28800000 exceeds 10% of

17/17 [==============================] - 13s 566ms/step - loss: 0.5737 - accuracy: 0.7091
Epoch 2/5
17/17 [==============================] - 9s 532ms/step - loss: 0.3253 - accuracy: 0.8471
Epoch 3/5
17/17 [==============================] - 9s 519ms/step - loss: 0.1754 - accuracy: 0.9333
Epoch 4/5
17/17 [==============================] - 9s 511ms/step - loss: 0.1300 - accuracy: 0.9394
Epoch 5/5
17/17 [==============================] - 9s 521ms/step - loss: 0.1636 - accuracy: 0.9515


In [14]:
Train_dataGeneration=ImageDataGenerator(rescale=1./255)
Train_batch=Train_dataGeneration.flow_from_directory(train_dir_model2c,target_size=image_size,batch_size=1,shuffle=False,seed=42,class_mode="categorical")

Train_batch.reset()
Train_result=model2c.evaluate_generator(Train_batch,steps=len(Train_batch),use_multiprocessing=False,verbose=1,workers=1)
print('Train loss:',Train_result[0],'Train accuracy:',Train_result[1])

prediction = model2c.predict_generator(Train_batch,steps=len(Train_batch))
predicted_class=np.argmax(prediction,axis=1)

print(confusion_matrix(Train_batch.classes,predicted_class))

Found 175 images belonging to 2 classes.


/tmp/ipykernel_7701/4024146043.py:5: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  Train_result=model2c.evaluate_generator(Train_batch,steps=len(Train_batch),use_multiprocessing=False,verbose=1,workers=1)
2023-08-15 17:02:47.343197: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


175/175 [==============================] - 12s 61ms/step - loss: 0.0482 - accuracy: 0.9943
Train loss: 0.048210807144641876 Train accuracy: 0.9942857027053833


/tmp/ipykernel_7701/4024146043.py:8: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  prediction = model2c.predict_generator(Train_batch,steps=len(Train_batch))
2023-08-15 17:02:58.879222: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


[[ 74   1]
 [  0 100]]


In [15]:
Test_dataGeneration=ImageDataGenerator(rescale=1./255)
Test_batch=Test_dataGeneration.flow_from_directory(test_dir_model2c,target_size=image_size,batch_size=1,shuffle=False,seed=42,class_mode="categorical")

Test_batch.reset()
Test_result=model2c.evaluate_generator(Test_batch,steps=len(Test_batch),use_multiprocessing=False,verbose=1,workers=1)
print('Test loss:',Test_result[0],'Test accuracy:',Test_result[1])

prediction = model2c.predict_generator(Test_batch,steps=len(Test_batch))
predicted_class=np.argmax(prediction,axis=1)

print(confusion_matrix(Test_batch.classes,predicted_class))

Found 50 images belonging to 2 classes.


/tmp/ipykernel_7701/3979543762.py:5: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  Test_result=model2c.evaluate_generator(Test_batch,steps=len(Test_batch),use_multiprocessing=False,verbose=1,workers=1)


 1/50 [..............................] - ETA: 7s - loss: 0.4925 - accuracy: 1.0000

2023-08-15 17:06:34.791043: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


50/50 [==============================] - 4s 69ms/step - loss: 0.0936 - accuracy: 0.9800
Test loss: 0.09361009299755096 Test accuracy: 0.9800000190734863


/tmp/ipykernel_7701/3979543762.py:8: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  prediction = model2c.predict_generator(Test_batch,steps=len(Test_batch))
2023-08-15 17:06:38.391776: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


[[24  1]
 [ 0 25]]
